In [1]:

import torch 
import argparse
import yaml
import time
import multiprocessing as mp
import torch.nn.functional as F
from tabulate import tabulate
from tqdm import tqdm
from torch.utils.data import DataLoader
from pathlib import Path
#from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from nmc.models import *
from nmc.datasets import * 
from nmc.augmentations import get_train_augmentation, get_val_augmentation
from nmc.losses import get_loss
from nmc.schedulers import get_scheduler
from nmc.optimizers import get_optimizer
from nmc.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate_epi
from nmc.utils.episodic_utils import * 
from scipy.cluster import hierarchy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn
from torch.optim import lr_scheduler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from scipy.cluster import hierarchy
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.data import Subset
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import cv2

In [2]:
with open('../configs/APTOS.yaml') as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)
print(cfg)
fix_seeds(3407)
setup_cudnn()
gpu = setup_ddp()
save_dir = Path(cfg['SAVE_DIR'])
save_dir.mkdir(exist_ok=True)
cleanup_ddp()

{'DEVICE': 'cuda:0', 'SAVE_DIR': 'output', 'MODEL': {'NAME': 'EfficientNetV2MModel', 'BACKBONE': 'EfficientNetV2', 'PRETRAINED': '/workspace/jhmoon/nmc_2024/checkpoints/pretrained/tf_efficientnetv2_m_weights.pth', 'UNFREEZE': 'full', 'VERSION': '384_32'}, 'DATASET': {'NAME': 'APTOSDataset', 'ROOT': '/data/public_data/aptos', 'TRAIN_RATIO': 0.7, 'VALID_RATIO': 0.15, 'TEST_RATIO': 0.15}, 'TRAIN': {'IMAGE_SIZE': [384, 384], 'BATCH_SIZE': 32, 'EPOCHS': 100, 'EVAL_INTERVAL': 25, 'AMP': False, 'DDP': False}, 'LOSS': {'NAME': 'CrossEntropy', 'CLS_WEIGHTS': False}, 'OPTIMIZER': {'NAME': 'adamw', 'LR': 0.001, 'WEIGHT_DECAY': 0.01}, 'SCHEDULER': {'NAME': 'warmuppolylr', 'POWER': 0.9, 'WARMUP': 10, 'WARMUP_RATIO': 0.1}, 'EVAL': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.APTOS.pth', 'IMAGE_SIZE': [384, 384]}, 'TEST': {'MODEL_PATH': 'checkpoints/pretrained/FGMaxxVit/FGMaxxVit.FGMaxxVit.APTOS.pth', 'FILE': 'assests/ade', 'IMAGE_SIZE': [384, 384], 'OVERLAY': True}}


In [3]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_score):
        if self.best_score is None:
            self.best_score = val_score
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_score
            self.counter = 0

In [4]:
def get_train_augmentation(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

def get_val_test_transform(size):
    return transforms.Compose([
        transforms.Resize(size),
        transforms.Lambda(lambda x: x.float() if x.dtype == torch.uint8 else x),
        transforms.Lambda(lambda x: x / 255.0 if x.max() > 1.0 else x),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [5]:
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        
        # 데이터셋에서 레이블 추출
        if hasattr(dataset, 'labels'):
            self.labels = dataset.labels
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        elif hasattr(dataset, 'targets'):
            self.labels = dataset.targets
            if isinstance(self.labels, np.ndarray):
                self.labels = torch.from_numpy(self.labels)
        else:
            try:
                self.labels = [sample[1] for sample in dataset]
                if isinstance(self.labels[0], np.ndarray):
                    self.labels = torch.from_numpy(np.array(self.labels))
                else:
                    self.labels = torch.tensor(self.labels)
            except:
                raise ValueError("Cannot access labels from dataset")
        
        self.n_classes = self.labels.shape[1] if len(self.labels.shape) > 1 else len(torch.unique(self.labels))
        self.samples_per_class = batch_size // self.n_classes
        
        # 클래스별 인덱스 저장
        self.class_indices = []
        for i in range(self.n_classes):
            if len(self.labels.shape) > 1:
                idx = torch.where(self.labels[:, i] == 1)[0]
            else:
                idx = torch.where(self.labels == i)[0]
            self.class_indices.append(idx)
        
        self.n_batches = len(self.dataset) // batch_size
        if len(self.dataset) % batch_size != 0:
            self.n_batches += 1
    
    def __iter__(self):
        for _ in range(self.n_batches):
            batch_indices = []
            for class_idx in range(self.n_classes):
                class_samples = self.class_indices[class_idx]
                if len(class_samples) == 0:
                    continue
                
                # 랜덤 선택
                selected = class_samples[torch.randint(len(class_samples), 
                                                     (self.samples_per_class,))]
                batch_indices.extend(selected.tolist())
            
            # 배치 크기에 맞게 자르기
            if len(batch_indices) > self.batch_size:
                batch_indices = batch_indices[:self.batch_size]
            
            # 중요: 리스트로 yield
            yield batch_indices
    
    def __len__(self):
        return self.n_batches

In [6]:
start = time.time()
best_mf1 = 0.0
device = torch.device(cfg['DEVICE'])
print("device : ", device)
num_workers = mp.cpu_count()
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = train_cfg['EPOCHS'], optim_cfg['LR']

image_size = [256,256]
image_dir = Path(dataset_cfg['ROOT']) / 'train_images'
train_transform = get_train_augmentation(image_size)
val_test_transform = get_val_test_transform(image_size)
batch_size = 32


dataset = eval(dataset_cfg['NAME'])(
    dataset_cfg['ROOT'] + '/combined_images',
    dataset_cfg['TRAIN_RATIO'],
    dataset_cfg['VALID_RATIO'],
    dataset_cfg['TEST_RATIO'],
    transform=None
)
trainset, valset, testset = dataset.get_splits()
trainset.transform = train_transform
valset.transform = val_test_transform
testset.transform = val_test_transform

# DataLoader 수정
trainloader = DataLoader(
    trainset, 
    batch_sampler=BalancedBatchSampler(trainset, batch_size=batch_size),
    num_workers=num_workers,
    pin_memory=True
)
valloader = DataLoader(valset, batch_size=1, num_workers=1, pin_memory=True)
testloader = DataLoader(testset, batch_size=1, num_workers=1, pin_memory=True)


device :  cuda:0
/data/public_data/aptos/combined_images
0    1263
2     699
1     259
4     207
3     135
Name: diagnosis, dtype: int64
Train size: 2563
0    271
2    150
1     55
4     44
3     29
Name: diagnosis, dtype: int64
Validation size: 549
0    271
2    150
1     56
4     44
3     29
Name: diagnosis, dtype: int64
Test size: 550


In [7]:
class WeightedFeatureFusion(nn.Module):
    def __init__(self, num_inputs):
        super().__init__()
        self.weights = nn.Parameter(torch.ones(num_inputs, dtype=torch.float32), requires_grad=True)
        
    def forward(self, features):
        # weights를 softmax로 정규화
        normalized_weights = F.softmax(self.weights, dim=0)
        # 가중치를 적용하고 합산
        weighted_sum = sum(w * f for w, f in zip(normalized_weights, features))
        return weighted_sum


In [8]:
class BiFPNBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        
        # Top-down 경로의 가중치 합산 layers
        self.td_weights = nn.ModuleList([
            WeightedFeatureFusion(2) for _ in range(4)  # P6->P3까지 4개의 fusion
        ])
        
        # Bottom-up 경로의 가중치 합산 layers
        self.bu_weights = nn.ModuleList([
            WeightedFeatureFusion(3) for _ in range(3)  # P4->P6까지 3개의 fusion
        ])
        self.bu_weights.append(WeightedFeatureFusion(2))  # P7은 2개 input
        
        # Convolution layers 수정
        self.conv_td = nn.ModuleList([
            nn.Sequential(
                # Depthwise convolution의 groups를 channels로 설정
                nn.Conv2d(channels, channels, 3, padding=1, groups=channels),
                # Pointwise convolution으로 채널 간 정보 교환
                nn.Conv2d(channels, channels, 1),
                nn.BatchNorm2d(channels),
                nn.ReLU()
            ) for _ in range(5)  # P7->P3
        ])
        
        self.conv_bu = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(channels, channels, 3, padding=1, groups=channels),
                nn.Conv2d(channels, channels, 1),
                nn.BatchNorm2d(channels),
                nn.ReLU()
            ) for _ in range(5)  # P3->P7
        ])
        
    def _resize_features(self, source, target):
        target_size = target.shape[-2:]
        return F.interpolate(source, size=target_size, mode='nearest')
    
    def forward(self, features):
        # features는 P3->P7 순서로 정렬된 리스트
        P3_in, P4_in, P5_in, P6_in, P7_in = features
        
        # Top-down 경로
        P7_td = self.conv_td[0](P7_in)
        
        P6_td_inputs = [P6_in, self._resize_features(P7_td, P6_in)]
        P6_td = self.conv_td[1](self.td_weights[0](P6_td_inputs))
        
        P5_td_inputs = [P5_in, self._resize_features(P6_td, P5_in)]
        P5_td = self.conv_td[2](self.td_weights[1](P5_td_inputs))
        
        P4_td_inputs = [P4_in, self._resize_features(P5_td, P4_in)]
        P4_td = self.conv_td[3](self.td_weights[2](P4_td_inputs))
        
        P3_td_inputs = [P3_in, self._resize_features(P4_td, P3_in)]
        P3_out = self.conv_td[4](self.td_weights[3](P3_td_inputs))
        
        # Bottom-up 경로
        P3_bu = P3_out
        
        P4_bu_inputs = [P4_in, P4_td, self._resize_features(P3_bu, P4_in)]
        P4_out = self.conv_bu[0](self.bu_weights[0](P4_bu_inputs))
        
        P5_bu_inputs = [P5_in, P5_td, self._resize_features(P4_out, P5_in)]
        P5_out = self.conv_bu[1](self.bu_weights[1](P5_bu_inputs))
        
        P6_bu_inputs = [P6_in, P6_td, self._resize_features(P5_out, P6_in)]
        P6_out = self.conv_bu[2](self.bu_weights[2](P6_bu_inputs))
        
        P7_bu_inputs = [P7_in, self._resize_features(P6_out, P7_in)]
        P7_out = self.conv_bu[3](self.bu_weights[3](P7_bu_inputs))
        
        return [P3_out, P4_out, P5_out, P6_out, P7_out]

In [9]:
class DualEfficientNetBiFPN(nn.Module):
    def __init__(self, efficientnet_nmc, efficientnet_aptos, num_classes=5, num_bifpn_blocks=3):
        super().__init__()
        
        # EfficientNet 모델들 저장 및 freeze
        self.nmc_model = efficientnet_nmc
        self.aptos_model = efficientnet_aptos
        
        # Freeze EfficientNet models
        for param in self.nmc_model.parameters():
            param.requires_grad = False
        for param in self.aptos_model.parameters():
            param.requires_grad = False
        
        # 주요 블록들의 출력 채널 수 확인
        main_blocks = self.get_main_blocks(efficientnet_nmc)
        
        def find_out_channels(block):
            modules = list(block.modules())
            for module in reversed(modules):
                if isinstance(module, nn.Conv2d):
                    return module.out_channels
            return None

        # 각 블록의 채널 수 저장
        self.block_channels = [find_out_channels(block) for block in main_blocks]
        print("Original block channels:", self.block_channels)
        
        # BiFPN에서 사용할 채널 수 (모든 레벨에서 동일하게 사용)
        self.bifpn_channels = 160  # 적절한 채널 수로 설정
        
        # Feature fusion layers
        self.fusion_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels * 2, self.bifpn_channels, 1),
                nn.BatchNorm2d(self.bifpn_channels),
                nn.ReLU()
            ) for in_channels in self.block_channels
        ])
        
        # BiFPN blocks
        self.bifpn_blocks = nn.ModuleList([
            BiFPNBlock(self.bifpn_channels) for _ in range(num_bifpn_blocks)
        ])
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.BatchNorm1d(self.bifpn_channels * 5),
            nn.Linear(self.bifpn_channels * 5, num_classes)
        )

        # Print parameter counts
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"Total parameters: {total_params}")
        print(f"Trainable parameters: {trainable_params}")
        print(f"Frozen parameters: {total_params - trainable_params}")
    
    @staticmethod
    def get_main_blocks(model):
        return [
            model.features[i] for i in range(1, 6)  # P3-P7에 해당하는 5개 레벨
        ]
    
    def extract_features(self, x, model):
        features = []
        # 먼저 초기 conv layer 통과
        x = model.features[0](x)
        
        # 각 블록을 통과하면서 feature 추출
        for block in self.get_main_blocks(model):
            x = block(x)
            features.append(x)
        return features
    
    def forward(self, x):
        # 각 모델에서 feature 추출
        nmc_features = self.extract_features(x, self.nmc_model)
        aptos_features = self.extract_features(x, self.aptos_model)
        
        # 각 레벨에서 feature concat 후 fusion
        combined_features = []
        for nmc_feat, aptos_feat, fusion_layer in zip(nmc_features, aptos_features, self.fusion_layers):
            # Concatenate features along channel dimension
            concat_feat = torch.cat([nmc_feat, aptos_feat], dim=1)
            # Apply fusion conv to adjust channels and combine features
            fused_feat = fusion_layer(concat_feat)
            combined_features.append(fused_feat)
        
        # BiFPN blocks 통과
        bifpn_features = combined_features
        for bifpn in self.bifpn_blocks:
            bifpn_features = bifpn(bifpn_features)
        
        # 모든 레벨의 feature를 결합하여 분류
        multi_scale_features = torch.cat([self.classifier[0](feat) for feat in bifpn_features], dim=1)
        out = self.classifier[1](multi_scale_features)  # Flatten
        out = self.classifier[2](out)  # BatchNorm
        out = self.classifier[3](out)  # Linear
        
        return out

In [10]:

def train_epoch(model, dataloader, criterion, optimizer, scaler, device):
    model.train()
    total_loss = 0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with autocast(enabled=scaler is not None):
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels.float())
        
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [11]:
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            # 각 클래스에 대해 시그모이드 적용 후 임계값 처리
            preds = (torch.sigmoid(outputs) > 0.5).int()
            
            # 배치 단위로 예측값과 라벨 저장
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # 배치 데이터를 하나의 배열로 결합
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # 멀티라벨 F1 score 계산
    f1 = f1_score(all_labels, all_preds, average='samples')  # or 'micro', 'macro', 'weighted'
    
    return f1

In [12]:
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, scaler, device, epochs):
    best_f1 = 0.0
    early_stopping = EarlyStopping(patience=10, min_delta=0.001)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
        val_f1 = evaluate(model, val_loader, device)
        
        print(f"Training Loss: {train_loss:.4f}")
        print(f"Validation F1 Score: {val_f1:.4f}")
        
        scheduler.step(val_f1)
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), 'model/fusion/best_model_combined_bifpn.pth')
            print("New best model saved!")
        
        early_stopping(val_f1)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
        
        print()
    
    return best_f1

In [13]:
# Model definition (changed to binary classification)
efficientnet_nmc = models.efficientnet_v2_m(pretrained=True)
num_ftrs = efficientnet_nmc.classifier[1].in_features
efficientnet_nmc.classifier = nn.Sequential(
    nn.BatchNorm1d(num_ftrs),
    nn.Linear(num_ftrs, 7)
)
efficientnet_nmc = efficientnet_nmc.to(device)

# Model definition (changed to binary classification)
efficientnet_aptos = models.efficientnet_v2_m(pretrained=True)
num_ftrs = efficientnet_aptos.classifier[1].in_features
efficientnet_aptos.classifier = nn.Sequential(
    nn.BatchNorm1d(num_ftrs),
    nn.Linear(num_ftrs, 5)
)
efficientnet_aptos = efficientnet_aptos.to(device)

/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/anaconda3/envs/gm/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Main execution code
# 정규화, lr스케쥴링, 데이터 증강, 조기종료, 배치정규화
epochs = 100

efficientnet_nmc.load_state_dict(torch.load('model/multilabel/best_model_nmc_cnn.pth'))
efficientnet_aptos.load_state_dict(torch.load('model/multilabel/best_model_aptos_cnn.pth'))

# 모델 생성
combined_model = DualEfficientNetBiFPN(
    efficientnet_nmc=efficientnet_nmc,
    efficientnet_aptos=efficientnet_aptos,
    num_classes=5,  # APTOS의 클래스 수에 맞춤
    num_bifpn_blocks=1
).to(device)

# L2 regularization
weight_decay = 1e-4
# Optimizer 생성 시 학습 가능한 파라미터만 전달
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, combined_model.parameters()),
    lr=0.0001, 
    weight_decay=weight_decay
)
criterion = nn.BCEWithLogitsLoss()
scaler = GradScaler(enabled=train_cfg['AMP'])
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)


best_f1 = train_and_evaluate(combined_model, trainloader, valloader, criterion, optimizer, scaler, device, epochs)

print(f"Training completed. Best F1 Score: {best_f1:.4f}")



Original block channels: [24, 48, 80, 160, 176]
Total parameters: 106178188
Trainable parameters: 440984
Frozen parameters: 105737204
Epoch 1/100


Evaluating: 100%|██████████| 549/549 [01:29<00:00,  6.15it/s]


Training Loss: 0.5801
Validation F1 Score: 0.6435
New best model saved!

Epoch 2/100


Evaluating: 100%|██████████| 549/549 [01:45<00:00,  5.19it/s]


Training Loss: 0.4993
Validation F1 Score: 0.7489
New best model saved!

Epoch 3/100


...

Training Loss: 0.0989
Validation F1 Score: 0.8130

Epoch 34/100


Evaluating: 100%|██████████| 549/549 [01:44<00:00,  5.24it/s]


Training Loss: 0.0986
Validation F1 Score: 0.8094

Epoch 35/100


Evaluating: 100%|██████████| 549/549 [01:42<00:00,  5.33it/s]

Training Loss: 0.1044
Validation F1 Score: 0.8075
Early stopping triggered
Training completed. Best F1 Score: 0.8160


In [14]:
# 모델 생성
combined_model = DualEfficientNetBiFPN(
    efficientnet_nmc=efficientnet_nmc,
    efficientnet_aptos=efficientnet_aptos,
    num_classes=5,  # APTOS의 클래스 수에 맞춤
    num_bifpn_blocks=1
).to(device)

Original block channels: [24, 48, 80, 160, 176]
Total parameters: 106178188
Trainable parameters: 440984
Frozen parameters: 105737204


In [15]:
# Final evaluation on test set
combined_model.load_state_dict(torch.load('model/fusion/best_model_combined_bifpn.pth'))
test_f1 = evaluate(combined_model, testloader, device)
print(f"Test F1 Score: {test_f1:.4f}")

Evaluating: 100%|██████████| 550/550 [01:27<00:00,  6.28it/s]

Test F1 Score: 0.8139


In [16]:
def evaluate(model, dataloader, device, num_classes):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            # 각 클래스에 대해 시그모이드 적용 후 임계값 처리
            preds = (torch.sigmoid(outputs) > 0.5).int()
            
            # 배치 단위로 예측값과 라벨 저장
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # 배치 데이터를 하나의 배열로 결합
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # 전체 F1 score 계산
    overall_f1 = f1_score(all_labels, all_preds, average='samples')
    
    # 각 클래스별 F1 score 계산
    class_f1_scores = f1_score(all_labels, all_preds, average=None)
    
    # 각 클래스별 정밀도(Precision)와 재현율(Recall) 계산
    class_precision = precision_score(all_labels, all_preds, average=None)
    class_recall = recall_score(all_labels, all_preds, average=None)
    
    # 결과를 딕셔너리로 정리
    results = {
        'overall_f1': overall_f1,
        'class_f1_scores': class_f1_scores,
        'class_precision': class_precision,
        'class_recall': class_recall
    }
    
    # 각 클래스별 메트릭 출력
    print("\nPer-class Performance Metrics:")
    print("-" * 50)
    for i in range(num_classes):
        print(f"Class {i}:")
        print(f"  F1-Score: {class_f1_scores[i]:.4f}")
        print(f"  Precision: {class_precision[i]:.4f}")
        print(f"  Recall: {class_recall[i]:.4f}")
    print("-" * 50)
    print(f"Overall F1-Score: {overall_f1:.4f}")
    
    return results

In [17]:
# Final evaluation on test set
combined_model.load_state_dict(torch.load('model/fusion/best_model_combined_bifpn.pth'))
test_f1 = evaluate(combined_model, testloader, device,5)

Evaluating: 100%|██████████| 550/550 [01:23<00:00,  6.56it/s]


Per-class Performance Metrics:
--------------------------------------------------
Class 0:
  F1-Score: 0.9778
  Precision: 0.9814
  Recall: 0.9742
Class 1:
  F1-Score: 0.5937
  Precision: 0.5278
  Recall: 0.6786
Class 2:
  F1-Score: 0.7586
  Precision: 0.7160
  Recall: 0.8067
Class 3:
  F1-Score: 0.3793
  Precision: 0.3793
  Recall: 0.3793
Class 4:
  F1-Score: 0.6429
  Precision: 0.6750
  Recall: 0.6136
--------------------------------------------------
Overall F1-Score: 0.8139
